In [1]:
import pandas as pd
import plotly.express as px
from fuzzywuzzy import process, fuzz
import numpy as np
from collections import Counter
import re 
import collections

### Merging Files into One Dataframe

Our dataset contains one CSV file per year so we read each file in as a dataframe and then concatenate the dataframe. The final dataset contains 10,529 entries spanning 2016 to 2023.

In [2]:
df_2016 = pd.read_csv('2016Export.csv', encoding_errors='ignore')
df_2016['year'] = 2016

df_2017 = pd.read_csv('2017Export.csv', encoding_errors='ignore')
df_2017['year'] = 2017

df_2018 = pd.read_csv('2018Export.csv', encoding_errors='ignore')
df_2018['year'] = 2018

df_2019 = pd.read_csv('2019Export.csv', encoding_errors='ignore')
df_2019['year'] = 2019

df_2020 = pd.read_csv('2020Export.csv', encoding_errors='ignore')
df_2020['year'] = 2020

df_2021 = pd.read_csv('2021Export.csv', encoding_errors='ignore')
df_2021['year'] = 2021

df_2022 = pd.read_csv('2022Export.csv', encoding_errors='ignore')
df_2022['year'] = 2022

df_2023 = pd.read_csv('2023Export.csv', encoding_errors='ignore')
df_2023['year'] = 2023

df = pd.concat([df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023], axis=0)


print(len(df))

10529


### Removing Entries with Incomplete Data

Next, we remove any entries that are missing an author block as this will prevent subsequent data cleaning. It looks like only one such entry is affected as the new total is 10,528 entries.

In [3]:
df.dropna(subset=['author_block'], inplace=True)
print(len(df))

10528


### Filtering Authors

We do data cleaning to separate the author portion and institution portion of the author block. We obtain the first and last authors in author block. We also remove middle initials so that each author is in the format F. Lastname (as opposed to F.M. Lastname). This is to make sure that authors are grouped properly even if their middle initial was used variably in the abstract. However, we do lose resolution if two authors with the same first initial and last name are merged (however this appears to be rare). We unfortunately do not have full first and last name information in this data.

In [4]:
df['author_block'] = df['author_block'].str.replace('(&nbsp;)', ' ')
df_author = df['author_block'].str.split('(<sup>,?[\d*]+(,?\.? ?[\d*]?)*<\/sup>)[\s\r\n]*[;]*[ ]*[\t]*(<sup>,?[\d*]*(,?\.? ?[\d*])*<\/sup>)', 
                                     expand=True)

df_author['authors'] = df_author[0].str.replace('(<sup>,?[0-9]+(,[0-9]+)*,?</sup>)', '')

df_author['author_list'] = df_author.apply(lambda x: x['authors'].split(','), axis=1)


df_author['first_author'] = df_author.apply(lambda x: x['author_list'][0], axis=1)
df_author['senior_author'] = df_author.apply(lambda x: x['author_list'][-1], axis=1)
df_author['senior_author2'] = df_author['senior_author'].str.replace('(. \w. )', '. ')



/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_19296/3902522543.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['author_block'] = df['author_block'].str.replace('(&nbsp;)', ' ')
/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_19296/3902522543.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_author['authors'] = df_author[0].str.replace('(<sup>,?[0-9]+(,[0-9]+)*,?</sup>)', '')
/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_19296/3902522543.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df_author['senior_author2'] = df_author['senior_author'].str.replace('(. \w. )', '. ')


### Filtering Institutions

For the institution list, we make the assumption that the first institution listed in the block is the main institution that contributed the work. In the string of the first assumption, we look for the words "university", "hospital", "institute", or "medical center" or if none of those words exist, the first expression separated by a comma. 

In [5]:

df_author['institutions'] = df_author[5].str.replace('(<sup>,?[0-9]+(,[0-9]+)*,?</sup>)', '|', n=1)

df_author['institution_list'] = df_author.apply(lambda x: x['institutions'].split('|'), axis=1)
df_author['first_institution'] = df_author.apply(lambda x: x['institution_list'][0].split(',')[:-3], axis=1)


KEYWORDS = ['university', 'hospital', 'institute', 'medical center']


def smart_institution(list):
    for item in list:
        for word in KEYWORDS:
            if word in item.lower():
                return item
    return list[0]
            

df_author['smart_institution'] = df_author.apply(lambda x: smart_institution(x['institution_list']), axis=1)
df_author['smart_institution2'] = df_author['smart_institution'].str.split(',', expand=True)[0]
df_author['smart_institution2'] = df_author['smart_institution2'].str.title()

# df_author = df_author[['first_author', 'senior_author', 'senior_author2', 'smart_institution2']]

# df_author

len(df_author)

/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_19296/2722030602.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_author['institutions'] = df_author[5].str.replace('(<sup>,?[0-9]+(,[0-9]+)*,?</sup>)', '|', n=1)


10528

### Combine Dataframes

Now, we combine the new columns into the existing dataframe.

In [6]:
df2 = pd.concat([df, df_author], axis=1)
df2

,control_number,session_number,session_type,abstract_title,author_block,abstract_body,abstract_type,abstract_scientific_area,abstract_clinical_area,year,...,authors,author_list,first_author,senior_author,senior_author2,institutions,institution_list,first_institution,smart_institution,smart_institution2
0,ASC20161024,1.01,QuickShot,Mimetic sHDL Nanoparticles: a Novel Drug-Deliv...,"P. T. White<sup>1</sup>, C. Subramanian<sup>1<...",<strong>Introduction</strong>: Triple negativ...,Basic Science,Experimental Therapeutics/Other,Cross Disciplinary,2016,...,"P. T. White, C. Subramanian, R. Kuai, J. Moon,...","[P. T. White, C. Subramanian, R. Kuai, J. M...",P. T. White,M. S. Cohen,M. Cohen,"University Of Michigan,Department Of Biomedica...","[University Of Michigan,Department Of Biomedic...","[University Of Michigan, Department Of Biomedi...","University Of Michigan,Department Of Biomedica...",University Of Michigan
1,ASC20160315,1.02,QuickShot,Novel Nano-Liposomes for Multi-Modal Image-Gui...,"P. Patel<sup>1</sup>, T. Kato<sup>1</sup>, H. ...",<strong>Introduction</strong>: To investigate ...,Basic Science,Experimental Therapeutics/Other,Cross Disciplinary,2016,...,"P. Patel, T. Kato, H. Ujiie, D. Lee, J. Ahn, H...","[P. Patel, T. Kato, H. Ujiie, D. Lee, J. A...",P. Patel,K. Yasufuku,K. Yasufuku,"University of Toronto,Division Of Thoracic Sur...","[University of Toronto,Division Of Thoracic Su...","[University of Toronto, Division Of Thoracic S...","University of Toronto,Division Of Thoracic Sur...",University Of Toronto
2,ASC20160574,1.03,QuickShot,Tumor-Targeting Nanotheranostic Micelles for N...,"R. Jaskula Sztul<sup>1,3</sup>, G. Chen<sup>2<...",<strong>Introduction</strong>: Although neuro...,Basic Science,Experimental Therapeutics/Other,Cross Disciplinary,2016,...,"R. Jaskula Sztul, G. Chen, A. Harrison, S. Gon...","[R. Jaskula Sztul, G. Chen, A. Harrison, S....",R. Jaskula Sztul,H. Chen,H. Chen,"University Of Wisconsin,Surgery,Madison, WI, U...","[University Of Wisconsin,Surgery,Madison, WI, ...","[University Of Wisconsin, Surgery]","University Of Wisconsin,Surgery,Madison, WI, USA",University Of Wisconsin
3,ASC20160105,1.04,QuickShot,SDF-1&alpha; Decreases Inflammation in Diabeti...,"M. M. Hodges<sup>1</sup>, C. Zgheib<sup>1</sup...","<strong>Introduction</strong>: In 2014, the U...",Basic Science,Experimental Therapeutics/Other,Cross Disciplinary,2016,...,"M. M. Hodges, C. Zgheib, J. Hu, J. Xu, K. W. L...","[M. M. Hodges, C. Zgheib, J. Hu, J. Xu, K....",M. M. Hodges,K. W. Liechty,K. Liechty,"University Of Colorado Denver,Laboratory For F...","[University Of Colorado Denver,Laboratory For ...","[University Of Colorado Denver, Laboratory For...","University Of Colorado Denver,Laboratory For F...",University Of Colorado Denver
4,ASC20161100,1.05,QuickShot,Mechanistic Analysis: Alcohol Induces Apoptosi...,"I. J. Lawandy<sup>1</sup>, B. A. Potz<sup>1</s...",<strong>Introduction</strong>: Autophagy serv...,Basic Science,Disease Pathogenesis,General Surgery,2016,...,"I. J. Lawandy, B. A. Potz, N. Y. Elmadhun, A. ...","[I. J. Lawandy, B. A. Potz, N. Y. Elmadhun, ...",I. J. Lawandy,F. W. Sellke,F. Sellke,"Brown University,Surgery/Cardiothoracic Surger...","[Brown University,Surgery/Cardiothoracic Surge...","[Brown University, Surgery/Cardiothoracic Surg...","Brown University,Surgery/Cardiothoracic Surger...",Brown University
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,ASC20230870,95.20,Quickshot,Exploring Barriers and Facilitators to Reducin...,"H. J. Petit<sup>1</sup>, G. A. Sullivan<sup>2<...",<strong>Introduction</strong>: \r\n\r\nThe ope...,Education,Other,General Surgery,2023,...,"H. J. Petit, G. A. Sullivan, S. M. Cocoma, J. ...","[H. J. Petit, G. A. Sullivan, S. M. Cocoma, ...",H. J. Petit,A. N. Shah,A. Shah,"Rush University Medical Center, Rush Medical C...","[Rush University Medical Center, Rush Medical ...","[Rush University Medical Center, Rush Medical...","Rush University 

In [65]:
len(df2['control_number'].unique())

10396

### Number of Abstracts by Year

First, we calculate the number of abstracts by year

In [46]:
df_years = df2.groupby('year').agg(
    total_count = ('control_number', 'nunique')
)

df_years.reset_index(inplace=True)
df_years
fig = px.bar(df_years, x='year', y='total_count', text_auto='',
labels = dict(year = 'Year', total_count = 'Total Abstracts (2016-2023)'))
# fig.show()

fig.update_layout(
    width=1400,
    height=800,
    font=dict(
        family="Inter",
        size=24))

fig.update_traces(textfont_size=24)

fig.update_yaxes(showticklabels=False)



fig.show()

fig.write_image('bar2.svg')



### Number of Abstracts By Institution

Here, we calculate the number of abstracts by institution for the duration of the study period

In [8]:
# df_institutions = df2.groupby(['smart_institution2', 'year']).agg(
#     count = ('control_number', 'nunique')
# )

df_institutions = df2.groupby(['smart_institution2']).agg(
    count = ('control_number', 'nunique')
)
df_institutions.sort_values(by='count', ascending=False, inplace=True)
df_institutions.reset_index(inplace=True)
df_institutions

,smart_institution2,count
0,University Of Alabama At Birmingham,477
1,University Of Michigan,348
2,University Of Wisconsin,179
3,Brigham And Women'S Hospital,166
4,Massachusetts General Hospital,165
...,...,...
1451,Naval Medical Center,1
1452,Nebraska Medical Center,1
1453,Nemour'S Children'S Hospital; University Of Ce...,1
1454,Nemours Alfred I Dupot Hospital For Children,1


### Correcting Institution Duplicates Using Fuzzy Matching

We need to combine data for institutions that are the same in reality even if the name is written differently. One example of this is "University of Alabama - Birmingham" vs "University of Alabama Birmingham" vs "University of Alabama". We will attempt to resolve this using fuzzy string matching. While not perfect, we can get pretty close with this and at least avoid errors among the most frequent institutions. Another example is "University of California" and "David Geffen School of Medicine". Given no string relation in this case, we will need to resolve these differences manually.


In [9]:
def _normalize_name(name) -> str:
    """
    Normalize the name of a university
    :param str name:
    :rtype: str
    """
    norm = name.lower()
    norm = re.sub(
        r'( +at +)|( *of *)|(university)|(hospital)|(medical center)|(institution)',
        ' ',
        norm,
    )
    return norm.replace("'", "").strip()


def process_unis(uni_list, uni_to_pub):
    """
    Process the list of unis
    :param List[str] uni_list:
    :param Dict[str, int] uni_to_pub:
    :rtype: Dict[str, str]
    """
    # Normalize unis
    uni_list_norm = [(uni, _normalize_name(uni)) for uni in uni_list]

    # Construct empty choices list - we will iterate through the
    # list of universities from highest to lowest number of publications
    # We'll normalize each one and then try to match it to the choices list
    #
    # If the normalized name is not in the choices list (i.e. there is
    # no match), we'll add it so that future universities can be matched to it
    # and map the original name to the normalized name
    #
    # If the normalized name is in the choices list, we'll map the original
    # name to the normalized name
    choices = []
    bad_to_good = collections.OrderedDict()
    for item, normalized_item in uni_list_norm:

        # if normalized name has cardinal directions in it or "washington", skip it
        if re.search(r'(north)|(south)|(east)|(west)|(central)|(washington)|(medical college)|(atlantic)|(commonwealth)|(children)|(illinois)|(atlantic)|(florida state)|(michigan state)|(mayo)|(york hospital)|(university college london)|(loyola)', normalized_item):
            bad_to_good[item] = normalized_item
            choices.append(normalized_item)
            continue

        potential_match = process.extractOne(normalized_item, choices)

        if not potential_match or potential_match[1] < 90:
            bad_to_good[item] = normalized_item
            choices.append(normalized_item)
        else:
            bad_to_good[item] = potential_match[0]

    # Now that we have a mapping of original name to normalized name
    # we can count the number of publications for each normalized name
    norm_counts = collections.defaultdict(int)
    for uni, norm in bad_to_good.items():
        norm_counts[norm] += uni_to_pub[uni]

    # Now we can go through the list of universities again and get the
    # normalized name from the bad_to_good mapping and add the number of
    # publications to the final map.
    #
    # Once we see a normalized name we'll add it to the seen_norms set
    # so that we don't double count and we'll assign the total number
    # of normalized publications to the most common name for that
    # university pre-normalization.
    #
    # Example - if we have Hospital of University of Pennsylvania (100) and
    # University of Pennsylvania Hospital (90), we'll assign the total number
    # of publications to Hospital of University of Pennsylvania (190) and
    # skip University of Pennsylvania Hospital (as they normalize to the same
    # thing)

    final_map = collections.defaultdict(int)
    norm_to_first_good_name = {}
    seen_norms = set()
    for uni in uni_list:
        norm = bad_to_good[uni]
        if norm not in seen_norms:
            norm_to_first_good_name[norm] = uni
            final_map[uni] += norm_counts[norm]
            seen_norms.add(norm)

    # print(norm_to_first_good_name)

    return {bad: norm_to_first_good_name.get(norm, None) for bad, norm in bad_to_good.items()}



l = df_institutions['smart_institution2'].to_list()
d = dict(zip(df_institutions['smart_institution2'], df_institutions['count']))

bad_to_good = process_unis(l, d)

bad_to_good

# # write a function to return a dict that contains all entries for which key != value
# def get_unequal_items(d):
#     return {k: v for k, v in d.items() if k != v}

# get_unequal_items(bad_to_good)

# clean_institution = process_unis(l, d)

# clean_institution

# df_institutions['smart_institution3'] = df_institutions.apply(
#     lambda x:
#     bad_to_good.get(x['smart_institution2'], None),
#     axis=1
# )

# df_institutions


# clean_institution




{'University Of Alabama At Birmingham': 'University Of Alabama At Birmingham',
 'University Of Michigan': 'University Of Michigan',
 'University Of Wisconsin': 'University Of Wisconsin',
 "Brigham And Women'S Hospital": "Brigham And Women'S Hospital",
 'Massachusetts General Hospital': 'Massachusetts General Hospital',
 'Stanford University': 'Stanford University',
 'Johns Hopkins University School Of Medicine': 'Johns Hopkins University School Of Medicine',
 'Indiana University School Of Medicine': 'Indiana University School Of Medicine',
 'Yale University School Of Medicine': 'Yale University School Of Medicine',
 'University Of Miami': 'University Of Miami',
 'University Of Texas Southwestern Medical Center': 'University Of Texas Southwestern Medical Center',
 'Medical College Of Wisconsin': 'Medical College Of Wisconsin',
 'University Of Florida': 'University Of Florida',
 'University Of Arizona': 'University Of Arizona',
 'University Of Colorado Denver': 'University Of Colorado De

In [10]:
df_bg = pd.DataFrame(bad_to_good.items(), columns=['smart_institution2', 'smart_institution3'])
df_bg.to_csv('maps.csv')

### Manual Cleaning

Here we will do manual string replacement for UCLA and Northwestern as this merge will have implications in the top 20. 

In [11]:
df3 = df2.merge(df_bg, on='smart_institution2')
df3['smart_institution3'].replace(['David Geffen School Of Medicine', 'Feinberg School Of Medicine - Northwestern University'], ['University Of California - Los Angeles', 'Northwestern University'], inplace=True)


# df_institutions2 = df3.groupby(['smart_institution3']).agg(
#     count = ('control_number', 'nunique')
# )
# df_institutions2.sort_values(by='count', ascending=False, inplace=True)
# df_institutions2.reset_index(inplace=True)
# df_institutions2[:50]

,smart_institution3,count
0,University Of Alabama At Birmingham,509
1,University Of Michigan,382
2,Hospital Of The University Of Pennsylvania,216
3,University Of Wisconsin,215
4,Johns Hopkins University School Of Medicine,202
5,University Of California - Los Angeles,197
6,Stanford University,185
7,Brigham And Women'S Hospital,176
8,University Of Miami,175
9,Yale University School Of Medicine,167


In [35]:
df_institution_rank = df3.groupby(['smart_institution3']).agg(
    count = ('control_number', 'nunique')
)


df_institution_rank.reset_index(inplace=True)
df_institution_rank.sort_values(by='count', ascending=False, inplace=True)

institution_subset = df_institution_rank['smart_institution3'].to_list()[:20]


fig = px.bar(df_institution_rank[0:20], x='smart_institution3', y='count', text='count',
labels = dict(total_count = 'Total Abstracts (2016-2023)'))


fig.update_layout(
    width=1400,
    height=800,
    font=dict(
        family="Inter",
        size=16))


fig.update_traces(textfont_size=18, textangle=0, textposition='auto')


# fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

fig.write_image('ranks.svg')



In [36]:
institution_subset

['University Of Alabama At Birmingham',
 'University Of Michigan',
 'Hospital Of The University Of Pennsylvania',
 'University Of Wisconsin',
 'Johns Hopkins University School Of Medicine',
 'University Of California - Los Angeles',
 'Stanford University',
 "Brigham And Women'S Hospital",
 'University Of Miami',
 'Yale University School Of Medicine',
 'Massachusetts General Hospital',
 'University Of Florida',
 'Indiana University School Of Medicine',
 'University Of Pittsburgh',
 'Ohio State University',
 'Northwestern University',
 'University Of Texas Southwestern Medical Center',
 'University Of Arizona',
 'Medical College Of Wisconsin',
 'University Of Colorado Denver']

### Number of Abstracts By Institution and Year

Here, we calculate the number of abstracts by institution during each year to look at trends over time

In [15]:
# df_institution_rank = df3.groupby(['smart_institution3', 'year']).agg(
#     count = ('abstract_title', 'count')
# )

# df_senior_author_rank = df3.groupby(['year', 'senior_author2']).agg(
#     count = ('abstract_title', 'count')
# )

# df_first_author_rank = df3.groupby(['year', 'first_author']).agg(
#     count = ('abstract_title', 'count')
# )

# df_first_author_rank = df3.groupby(['year', 'first_author']).agg(
#     count = ('abstract_title', 'count')
# )

# df_area_rank = df3.groupby(['year', 'abstract_clinical_area']).agg(
#     count = ('abstract_title', 'count')
# )

# df_institution_rank.reset_index(inplace=True)


# institutions = df_institutions['smart_institution3'][0:20].to_list()
# institutions

# df_institution_rank2 = df_institution_rank[df_institution_rank['smart_institution3'].isin(institutions)]

# df_new  df_institution_year_rank.groupby('smart_institution2').agg(
#     avg_abstracts = ('count', 'mean')
# )

# df_new.reset_index(inplace=True)

# df_new.sort_values(by='avg_abstracts', ascending=False, inplace=True)



# df_institution_year_rank['moving']  df_institution_year_rank.groupby(['smart_institution2'])['count'].transform(lambda x: x.rolling(10, 1).mean()
# df_institution_year_rank['avg']  df_institution_year_rank.groupby(['smart_institution2'])['count'].mean()

# df_institution_year_rank
# df_institution_year_rank  df_institution_ran[df_institution_year_rank['smart_institution2'].isin(institutions)

In [44]:
df_institution_year_rank = df3.groupby(['smart_institution3', 'year']).agg(
    count = ('control_number', 'nunique')
)

df_institution_year_rank.reset_index(inplace=True)

df_subset = df_institution_year_rank[df_institution_year_rank['smart_institution3'].isin(institution_subset)]
df_subset.sort_values(by=['year', 'count'], ascending=False, inplace=True)
df_subset

/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_19296/2565774084.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,smart_institution3,year,count
1724,University Of Alabama At Birmingham,2023,73
1946,University Of Michigan,2023,52
1777,University Of California - Los Angeles,2023,37
673,Hospital Of The University Of Pennsylvania,2023,35
916,Massachusetts General Hospital,2023,35
...,...,...,...
2139,University Of Texas Southwestern Medical Center,2016,10
1180,Northwestern University,2016,9
1210,Ohio State University,2016,8
1770,University Of California - Los Angeles,2016,8


In [47]:

fig = px.line(df_subset, x="year", y="count", color='smart_institution3',
labels = dict(year='', count='Number of Abstracts', smart_institution2 = 'Institution'))
fig.update_layout(width=1500, height=800)

# fig.for_each_trace(lambda t: t.update(name = d[t.name]))

fig.update_layout(
    width=1400,
    height=800,
    font=dict(
        family="Inter",
        size=16))

fig.update_traces(line=dict(width=5))

fig.write_image('years.svg')

fig.show()

### Year over Year Changes

Here, we look at data from 2021 to 2023 and measure the difference in abstracts from year-to-year in order to measure the absolute and per cent YoY changes in abstract acceptance.

In [49]:
# df_prev_value = df_institution_rank2.groupby(['smart_institution3', 'year'])['count'].shift(1)
# df_prev_value

df_changes = df_institution_year_rank


df_changes['prev_count'] = df_changes['count'].shift().where(df_changes['smart_institution3'].eq(df_changes['smart_institution3'].shift()))
df_changes


,smart_institution3,year,count,prev_count
0,A T Still University,2020,1,NaN
1,A T Still University,2021,1,1.0
2,Abilene Christian University,2019,1,NaN
3,Abington Jefferson Health,2017,1,NaN
4,Abington Jefferson Health,2018,2,1.0
...,...,...,...,...
2373,Yokohama City University Medical Center,2017,1,1.0
2374,Ysbyty Gwynedd Hospital,2020,1,NaN
2375,Zhongshan Hospital Fudan University,2017,2,NaN
2376,Zhongshan Hospital Fudan University,2018,1,2.0


In [62]:


df_changes['abs_change'] = df_changes['count']-df_changes['prev_count']
df_changes['percent_change'] = (df_changes['count']-df_changes['prev_count'])/df_changes['prev_count']*100
df_changes['inst_year'] = df_changes.apply(lambda x: ' '.join([x['smart_institution3'], str(x['year'])]), axis=1)

df_changes = df_changes[~df_changes['abs_change'].isna()]
df_changes

df_changes.sort_values(by='percent_change', ascending=False, inplace=True)

df_changes_subset = df_changes[(df_changes['year']>2021) &
                               (df_changes['smart_institution3'].isin(institution_subset))]

fig = px.bar(df_changes_subset, x='inst_year', y='percent_change', text_auto='',
labels = dict(inst_year= '', percent_change='Percent Change (YoY)'))

fig.update_yaxes(range=[-150, 550])
fig.update_traces(textfont_size=14, textangle=0, texttemplate='%{y:.0f}%', textposition='outside')


fig.update_layout(
    width=1400,
    height=800,
    font=dict(
        family="Inter",
        size=16))


fig.write_image('pct_change.svg')
        
fig.show()



In [61]:
df_changes_subset.sort_values(by='abs_change', ascending=False, inplace=True)

fig = px.bar(df_changes_subset, x='inst_year', y='abs_change', text_auto='',
labels = dict(inst_year= '', abs_change='Absolute Change (YoY)'))

# fig.update_layout(width=1200, height=1200)

fig.update_traces(textfont_size=16, textangle=0, texttemplate='%{y:.0f}', textposition='outside')


fig.update_yaxes(range=[-25, 35])

fig.update_layout(
    width=1400,
    height=800,
    font=dict(
        family="Inter",
        size=16))
fig.show()

fig.write_image('abs_change.svg')

/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_19296/1234533550.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_senior_author_rank.reset_index(inplace=True)
df_senior_author_rank.sort_values(by=['year', 'count'], ascending=False, inplace=True)
df_senior_author_rank


fig = px.line(df_senior_author_rank, x="year", y="count", color='senior_author2')
fig.update_layout(width=1400, height=800)
fig.show()
